In [ ]:
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.datasets import SinglePointLmdbDataset, TrajectoryLmdbDataset
import ase.io
from ase.build import bulk
from ase.build import fcc100, add_adsorbate, molecule
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.optimize import BFGS
import matplotlib.pyplot as plt
import lmdb
import pickle
from tqdm import tqdm
import torch
import os
from functions import *
from ase.db import connect
np.random.seed(42)

db_path = "./02-data_db/ads_ads-3s_ns3.db"
#split train:val:test = ?
split = [8,2,0]
lmdb_name = get_filename_from_path(db_path)

os.makedirs("./03-lmdb/"+lmdb_name+"/train/")
train_path = ("./03-lmdb/"+lmdb_name +"/train/"+lmdb_name+"train.lmdb")

os.makedirs("./03-lmdb/"+lmdb_name+"/val/")
val_path = ("./03-lmdb/"+lmdb_name+"/val/" +lmdb_name+"val.lmdb")

if split[2] != 0:
    os.makedirs("./03-lmdb/"+lmdb_name+"test/")
    test_path = ("./03-lmdb/"+lmdb_name+"test/"+lmdb_name+"test.lmdb")

db = connect(db_path)
a2g = AtomsToGraphs(
    max_neigh=50,
    radius=6,
    r_energy=True,    # False for test data
    r_forces=True,    # False for test data
    r_distances=True,
    r_fixed=True,
)

data_a2g = a2g.convert_all(db, disable_tqdm=True)
sid_set, fid_set = get_sid_fid_from_db(db)


data_a2g, sid_set, fid_set = shuft_a2g_sid_fid(data_a2g, sid_set, fid_set)

train_a2g, val_a2g, test_a2g = split_array(data_a2g,split)
train_sid, val_sid, test_sid = split_array(sid_set,split)
train_fid, val_fid, test_fid = split_array(fid_set,split)

train_a2g, train_sid, train_fid = shuft_a2g_sid_fid(train_a2g, train_sid, train_fid)
val_a2g, val_sid, val_fid = shuft_a2g_sid_fid(val_a2g, val_sid, val_fid)
test_a2g, test_sid, test_fid = shuft_a2g_sid_fid(test_a2g, test_sid, test_fid)

train_ = write_lmdb(train_path, train_a2g, train_sid, train_fid)
val_ = write_lmdb(val_path, val_a2g, val_sid, val_fid )
if split[2] != 0 : 
    test_ = write_lmdb(test_path, test_a2g, test_sid, test_fid)
dataset = TrajectoryLmdbDataset({"src": train_path})
len(dataset)

100%|██████████| 86/86 [00:00<00:00, 667.89it/s]


343